# Log : PowerBI Usage

In [1]:
import msal
import requests
import json
import pandas as pd
from datetime import date, timedelta
import os
from dotenv import load_dotenv

load_dotenv()

class config:
    client_id =  os.getenv("client_id")
    client_secret = os.getenv("client_secret")
    authority_url = os.getenv("authority_url").strip("'")
    scope = ['https://analysis.windows.net/powerbi/api/.default']
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    mssql_server =  os.getenv("mssql_server")
    mssql_db = os.getenv("mssql_db")
    mssql_user = os.getenv("mssql_user")
    mssql_pw = os.getenv("mssql_pw")
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    hana_host = os.getenv("hana_host")
    hana_port = os.getenv("hana_port")
    hana_user = os.getenv("hana_user")
    hana_pw = os.getenv("hana_pw")


In [2]:

# loop previous 30 days
for day in range(1,30):
    activityDate = (date.today() - timedelta(days=day)).strftime("%Y-%m-%d")
    url = "https://api.powerbi.com/v1.0/myorg/admin/activityevents?startDateTime='" + activityDate + "T00:00:00'&endDateTime='" + activityDate + "T23:59:59'"

    #Use MSAL to grab token
    app = msal.ConfidentialClientApplication(config.client_id,
                                             authority= f'https://login.microsoftonline.com/{config.authority_url}',
                                             client_credential= config.client_secret)
    result = app.acquire_token_for_client(scopes= config.scope)

    #Get latest Power BI Activities
    if 'access_token' in result:
        access_token = result['access_token']
        header = {'Content-Type':'application/json', 'Authorization':f'Bearer {access_token}'}
        api_call = requests.get(url=url, headers=header)

        #Specify empty Dataframe with all columns
        column_names = ['Id', 'RecordType', 'CreationTime', 'Operation', 'OrganizationId', 'UserType', 'UserKey', 'Workload', \
                        'UserId', 'ClientIP', 'UserAgent', 'Activity', 'IsSuccess', 'RequestId', 'ActivityId', 'ItemName', 'WorkSpaceName', \
                        'DatasetName', 'ReportName', 'WorkspaceId', 'ObjectId', 'DatasetId', 'ReportId', 'ReportType', 'DistributionMethod', 'ConsumptionMethod']
        df = pd.DataFrame(columns=column_names)

        #Set continuation URL
        contUrl = api_call.json()['continuationUri']
        
        #Get all Activities for first hour, save to dataframe (df1) and append to empty created df
        result = api_call.json()['activityEventEntities']
        df1 = pd.DataFrame(result)
        pd.concat([df, df1])

        #Call Continuation URL as long as results get one back to get all activities through the day
        while contUrl is not None:        
            api_call_cont = requests.get(url=contUrl, headers=header)
            contUrl = api_call_cont.json()['continuationUri']
            result = api_call_cont.json()['activityEventEntities']
            df2 = pd.DataFrame(result)
            df = pd.concat([df, df2])
        
        #Set ID as Index of df
        df = df.set_index('Id')
        
    #Save df as CSV
    path = r'C:\Users\siravit.pim\Desktop\forBI/'
    df.to_csv(path + activityDate + '.csv')
    
    print(f"processing {activityDate}")
print("done !")

processing 2024-08-04
processing 2024-08-03
processing 2024-08-02
processing 2024-08-01
processing 2024-07-31
processing 2024-07-30
processing 2024-07-29
processing 2024-07-28
processing 2024-07-27
processing 2024-07-26
processing 2024-07-25
processing 2024-07-24
processing 2024-07-23
processing 2024-07-22
processing 2024-07-21
processing 2024-07-20
processing 2024-07-19
processing 2024-07-18
processing 2024-07-17
processing 2024-07-16
processing 2024-07-15
processing 2024-07-14
processing 2024-07-13
processing 2024-07-12
processing 2024-07-11
processing 2024-07-10
processing 2024-07-09
processing 2024-07-08
processing 2024-07-07
done !
